## Importing Libraries,setting working directory and reading data.

In [18]:
#Importing libraries

import os
import pandas as pd # data manipulation
import numpy as np # linear Algebra and mathematics
from sklearn.tree import DecisionTreeRegressor # importing the Decision Tree class
from sklearn.ensemble import RandomForestRegressor # importing the RandomForest class
from sklearn import preprocessing # help in data impuation(NaNs) and Categorical features encoding
from sklearn.tree import export_graphviz # This function generates a GraphViz representation of the decision tree, which is then written into out_file
import datetime as dt # will be useful to convert the date to number of days since some point
import matplotlib.pyplot as plt
%matplotlib inline
import six # Six is a Python 2 and 3 compatibility library

#Setting working Directory
os.chdir('C:/Users/manum/Desktop/Daily Study/Kaggle_Competitions/2.Sberbank Russian Housing Market - House Price Prediction/Data')
print('Working Directory :','\n',os.getcwd(),'\n')

#Reading data
df_train=pd.read_csv('C:/Users/manum/Desktop/Daily Study/Kaggle_Competitions/2.Sberbank Russian Housing Market - House Price Prediction/Data/train.csv')
df_test=pd.read_csv('C:/Users/manum/Desktop/Daily Study/Kaggle_Competitions/2.Sberbank Russian Housing Market - House Price Prediction/Data/test.csv')
df_macro=pd.read_csv('C:/Users/manum/Desktop/Daily Study/Kaggle_Competitions/2.Sberbank Russian Housing Market - House Price Prediction/Data/macro.csv')


print('Shape of the data : ', df_train.shape)
df_train.head()

Working Directory : 
 C:\Users\manum\Desktop\Daily Study\Kaggle_Competitions\2.Sberbank Russian Housing Market - House Price Prediction\Data 

Shape of the data :  (30471, 292)


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


### Data preparation

* Create a vector containing the ids for our predictions
* Create a vector of the target variables in the training set
* Create joint train and test set to make data wrangling quicker and consistent on train and test
* Removing the id (could it be a useful source of leakage?)
* Convert the date into a number (of days since some point)
* Deal with categorical variables
* Deal with missing values

In [19]:
# Create a vector containing the id's for our predictions
id_test=df_test.id

#Create a vector of the target variables in the training set
# Transform target variable so that loss function is correct (ie we will use RMSLE in place of RMSE)
# ylog1p_train will be log(1+y),
"""
This transformation is primarily used to convert the non-linear target variable into linear target variable, so that the linear 
model works well with the data and the performance of the model with transformed target improves

"""

ylog1p_train = np.log1p(df_train['price_doc'].values)
df_train = df_train.drop(["price_doc"], axis=1)

# combine both training and test sets to make data wrangling quicker and consistent on train and test sets
df_train['TrainOrTest']='Train'
df_test['TrainOrTest']='Test'

df_all=pd.concat([df_train,df_test])

# Remove ID column
df_all=df_all.drop('id',axis=1)


In [34]:
df_all['timestamp']=pd.to_datetime(df_all['timestamp'])
fromDate = min(df_all['timestamp'])
df_all['timedelta'] = (df_all['timestamp'] - fromDate).dt.days.astype(int)
print(df_all[['timestamp', 'timedelta']].head())
df_all.drop('timestamp', axis = 1, inplace = True)

   timestamp  timedelta
0 2011-08-20          0
1 2011-08-23          3
2 2011-08-27          7
3 2011-09-01         12
4 2011-09-05         16


### Encoding Categorical Features

We will take a naive approach and assign a numeric value to each categorical feature in our training and test sets. Sklearn's preprocessing unit has a tool called LabelEncoder() which can do just that for us.

In [42]:
for c in df_all.columns:
    if df_all[c].dtype=='object':
        lbl=preprocessing.LabelEncoder()
        lbl.fit(df_all[c].tolist())
        df_all[c]=lbl.transform(df_all[c].tolist())

### Addressing problems with NaN in the data

As we saw from our EDA there were quite a lot of NaN in the data. Our model won't know what to do with these so we need to replace them with something sensible.

There are quite a few options we can use - the mean, median, most_frequent, or a numeric value like 0. Playing with these will give different results, for now I have it set to use the mean.

This uses the mean of the column in which the missing value is located.

In [61]:
# Number of missing value counts by column
missing_val_count_by_column=(df_all.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

"""
df_missing=df_all.isnull().sum(axis=0).reset_index()
df_missing.columns=['ColumnName','MissingValuesCount']
idx_ = df_missing['MissingValuesCount']>0

"""

#Get names of categorical columns with missing values  
cat_cols_with_missing=[col for col in df_all.columns
                  if df_all[col].isnull().any() and df_all[col].dtype=='object']



#Get names of numerical columns with missing values 

num_cols_with_missing=[col for col in df_all.columns
                  if df_all[col].isnull().any() and df_all[col].dtype!='object']


life_sq                                   7559
floor                                      167
max_floor                                 9572
material                                  9572
build_year                               14654
num_room                                  9572
kitch_sq                                  9572
state                                    14253
preschool_quota                           8284
school_quota                              8280
hospital_beds_raion                      17859
raion_build_count_with_material_info      6209
build_count_block                         6209
build_count_wood                          6209
build_count_frame                         6209
build_count_brick                         6209
build_count_monolith                      6209
build_count_panel                         6209
build_count_foam                          6209
build_count_slag                          6209
build_count_mix                           6209
raion_build_c

### Imputing the missing values

In [87]:
# Instantiate an imputer

"""
x['Age'].fillna(x['Age'].mean(), inplace=True) #single varaible can be fixed this way

"""

from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values='NaN', strategy = 'mean')

# Replace infinite values by NaN, before imputing
#df_all.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fit the imputer using all of our data (but not any dates)
df_all=pd.DataFrame(imputer.fit_transform(df_all))



# imputation removed column names; put them back



ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [92]:
np.where(df_all.values >= np.finfo(np.float64).max)

C:\Users\manum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater_equal
  """Entry point for launching an IPython kernel.


(array([], dtype=int64), array([], dtype=int64))